In [ ]:
%matplotlib widget

# загрузим данные
import numpy as np
import pandas as pd

file_path = "../npz/BTCUSD_1T_close_only.npz"
data = np.load(file_path, allow_pickle=True)
df = pd.DataFrame(data['data'], columns=['timestamp', 'close'])
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.set_index('timestamp', inplace=True)



In [ ]:
dfc = df[:1000].copy()

In [ ]:
lim = 100
def get_transitions_map(df, windows, tau, grid_size):
    """
    creates a map of price shifts for each grid point
    """
    max_window = max(windows)
    for i in range(lim):
        dfc = df[i:max_window+i].copy()


In [ ]:
from scipy.signal import savgol_coeffs
import numpy as np

def savgol_last_only(data, window_size=51, polyorder=3):
    """
    Рассчитывает Savitzky-Golay фильтр без заглядывания в будущее,
    но использует только последнюю точку в окне, оптимизируя вычисления.
    
    :param data: массив входных данных
    :param window_size: размер окна фильтра
    :param polyorder: порядок полинома
    :return: массив сглаженных значений (только последняя точка окна)
    """
    # Получаем коэффициенты свёртки для последней точки в окне
    coeffs = savgol_coeffs(window_size, polyorder, deriv=0, use="conv")[-1::-1]  # Разворачиваем, чтобы применять к последним точкам

    # Применяем только к последним `window_size` точкам в каждом окне
    result = np.array([
        np.dot(data[i - window_size + 1:i + 1], coeffs)
        for i in range(window_size - 1, len(data))
    ])

    # Дополним начало массива NaN, так как первые `window_size - 1` значений нельзя посчитать
    result = np.concatenate([np.full(window_size - 1, np.nan), result])

    return result


In [ ]:
filtered = savgol_last_only(dfc['close'].values, window_size=51, polyorder=3)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
#plt.plot(dfc['close'], label='original')
plt.plot(dfc['close'], label='dfc')
plt.plot(dfc.index, filtered, label='fixed')

In [ ]:
from scipy.signal import savgol_filter

# Функция в лоб: без оптимизации, но без заглядывания в будущее
def savgol_last_naive(data, window_size=51, polyorder=3):
    """
    Рассчитывает Savitzky-Golay фильтр без заглядывания в будущее,
    проходя окном по данным и беря последнюю точку фильтрации.
    
    :param data: массив входных данных
    :param window_size: размер окна фильтра
    :param polyorder: порядок полинома
    :return: массив сглаженных значений (только последняя точка окна)
    """
    result = []
    for i in range(len(data) - window_size + 1):
        # Применяем Savitzky-Golay к текущему окну
        smoothed_window = savgol_filter(data[i:i + window_size], window_size, polyorder, mode='nearest')
        result.append(smoothed_window[-1])  # Берем последнюю точку

    # Дополним начало массива NaN, так как первые `window_size - 1` значений нельзя посчитать
    result = np.concatenate([np.full(window_size - 1, np.nan), result])

    return np.array(result)


In [ ]:
filtered2 = savgol_last_naive(dfc['close'].values, window_size=51, polyorder=3)

In [ ]:
#%pip install pykalman
#%pip install ta-lib
#%pip install pandas-ta

In [ ]:
import scipy.signal
def causal_gaussian_filter(series, sigma):
    """ Применяет каузальный Гауссов фильтр (без заглядывания в будущее) """
    # Генерируем Гауссовое ядро, но берём только левую половину
    window_size = int(4 * sigma)  # Длина окна
    kernel = np.exp(-np.linspace(0, window_size, window_size)**2 / (2 * sigma**2))
    kernel /= np.sum(kernel)  # Нормируем, чтобы сумма весов = 1

    # Свёртка с учетом только прошлых значений (mode='valid' обрезает будущее)
    filtered = scipy.signal.convolve(series, kernel, mode='full')[:len(series)]
    return filtered

In [ ]:
import statsmodels.api as sm

def causal_hp(data, window_size=51, lamb=80):
    result = []
    for i in range(len(data) - window_size + 1):
        smoothed_window = sm.tsa.filters.hpfilter(data[i:i + window_size], lamb=lamb)[1]
        result.append(smoothed_window.iloc[-1])

    # Дополним начало массива NaN, так как первые `window_size - 1` значений нельзя посчитать
    result = np.concatenate([np.full(window_size - 1, np.nan), result])

    return np.array(result)


In [ ]:

import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d
import statsmodels.api as sm
from pykalman import KalmanFilter
import pandas_ta as ta


def ppp(i=0):
    start = 100 + i
    end = start + 800
    dfc = df[start:end].copy()
    plt.figure(figsize=(12, 6))
    #plt.plot(dfc['close'], label='original')
    plt.plot(dfc['close'], label='dfc', color='gray')
    #plt.plot(dfc.index, savgol_last_naive(dfc['close'].values, window_size=51, polyorder=3), label='savgol_last_naive', color='red')
    
    #plt.plot(dfc.index, filtered, label='fixed', color='red')
    #plt.plot(dfc.index, filtered2, label='fixed', color='blue')
    plt.plot(dfc.index,   savgol_filter(dfc['close'], 51, 3, mode='nearest')     , label='savgol_filter', color='red')
    sig = 51/4.71/2
    plt.plot(dfc.index, gaussian_filter1d(dfc['close'].astype(float).values, sigma=sig), label=f'gaussian_filter1d sig={sig}', color='blue')
    #plt.plot(dfc.index, sm.tsa.filters.hpfilter(dfc, lamb=80)[1], label='hpfilter', color='blue')
    #plt.plot(dfc.index, causal_hp(dfc, window_size=500, lamb=1080), label='causal_hp', color='red') 
    # kf = KalmanFilter(initial_state_mean=dfc['close'].iloc[0], n_dim_obs=1)
    # kf_result, _ = kf.smooth(dfc['close'].values)
    # plt.plot(dfc.index, kf_result, label='kf_result', color='blue') 
    # fft = np.fft.rfft(dfc['close'].values)
    # fft[25:] = 0  # Оставляем только первые 10 частот
    # plt.plot(dfc.index, np.fft.irfft(fft, n=len(dfc)), label='fft', color='blue')
    #plt.plot(dfc.index, talib.WMA(dfc['close'], timeperiod=50), label='fft', color='blue')
    #plt.plot(dfc.index, ta.wma(dfc['close'], length=30).shift(0), label='wma', color='blue')
    #plt.plot(dfc.index, ta.wma(dfc['close'], length=30).shift(-10), label='wma', color='red')
    #plt.plot(dfc.index, dfc['close'].ewm(span=30, adjust=False).mean(), label='ema30', color='red', linestyle='--')
    # plt.plot(dfc.index, dfc['close'].ewm(span=30, adjust=False).mean().shift(-10), label='ema30', color='blue')
    #plt.plot(dfc.index, dfc['close'].ewm(span=60, adjust=False).mean().shift(-15), label='ema60', color='red')
    # plt.plot(dfc.index, dfc['close'].ewm(span=120, adjust=False).mean().shift(-20), label='ema60', color='green')
    # sig = 6
    # plt.plot(dfc.index, causal_gaussian_filter(dfc['close'], sigma=sig), label=f'causal_gaussian_filter sig={sig}', color='blue')
    plt.legend()
    

i=0
ppp()

In [ ]:
i+=10
ppp(i)